In [1]:
# import packages
import pandas as pd
import numpy as np

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

/Users/abdygaziev/anaconda3/envs/learn-deep/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/abdygaziev/anaconda3/envs/learn-deep/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/abdygaziev/anaconda3/envs/learn-deep/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/abdygaziev/a

In [2]:
!dir

/bin/sh: dir: command not found


In [3]:
!pwd

/Users/abdygaziev/Documents/FlatironMaterials/Projects/fraud_detection_neuralnet


In [22]:
df = pd.read_pickle('/Users/abdygaziev/Documents/FlatironMaterials/Projects/train_dummy.pickle')

In [23]:
# from this data we get our target data
df.head(5)

,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,C1,...,P_emaildomain_web.de,P_emaildomain_windstream.net,P_emaildomain_yahoo.co.jp,P_emaildomain_yahoo.co.uk,P_emaildomain_yahoo.com,P_emaildomain_yahoo.com.mx,P_emaildomain_yahoo.de,P_emaildomain_yahoo.es,P_emaildomain_yahoo.fr,P_emaildomain_ymail.com
TransactionID,,,,,,,,,,,,,,,,,,,,,
3001687,0,413891,45.000,16661,490.0,150.0,226.0,327.000000,87.00000,1.0,...,0,0,0,0,0,0,0,0,0,0
3001688,0,413902,50.000,6194,498.0,150.0,117.0,337.000000,87.00000,145.0,...,0,0,0,0,0,0,0,0,0,0
3001689,1,413915,160.475,9635,405.0,185.0,226.0,290.733794,86.80063,1.0,...,0,0,0,0,0,0,0,0,0,0
3001690,0,413942,50.000,11207,361.0,150.0,226.0,126.000000,87.00000,1.0,...,0,0,0,0,0,0,0,0,0,0
3001692,0,413957,50.000,12544,321.0,150.0,226.0,476.000000,87.00000,1.0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
df.shape

(533007, 267)

In [24]:
# train test split
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

341124 train examples
85281 validation examples
106602 test examples
